# Analysis of linear models between ECoG, SYLBER embeddings

In [5]:
import os
import re
import numpy as np
import scipy as sp
import sys
sys.path.append('../')
from utils import data_utils, stat_utils
import matplotlib.pyplot as plt

In [2]:
def search_trf_files(
    sub_num,
    task=None,
    mode=None,
    base_dir='../TRF_outputs/',
    file_ext=".npy"):
    sub_str = f"sub{int(sub_num):02d}"
    sub_dir = os.path.join(base_dir, sub_str)
    if not os.path.isdir(sub_dir):
        raise FileNotFoundError(f"Subject directory {sub_dir} not found.")

    pattern = rf"{sub_str}_task(?P<task>\w+)_mode(?P<mode>\w+)_.*\.pickle_preds\.pickle{re.escape(file_ext)}$"

    matches = []
    for fname in os.listdir(sub_dir):
        if not fname.endswith(file_ext):
            continue
        match = re.match(pattern, fname)
        if match:
            if (task is None or match.group("task") == task) and \
               (mode is None or match.group("mode") == mode):
                matches.append(os.path.join(sub_dir, fname))

    return matches

search_trf_files(1, task='decode', mode='gamma')

['../TRF_outputs/sub01/sub01_taskdecode_modegamma_r0.5_min-0.5_max0.1_f50_a2.15e+06.pickle_preds.pickle.npy']

# DECODING

In [3]:
GT = np.load('../TRF_outputs/ground_truth_r05.npy')
SYLBER_FEATURES = data_utils.SYLBER_FEATURES
SEGMENTS = SYLBER_FEATURES['segments']
START_TIME = 9000

In [ ]:
modes = ('alpha', 'beta', 'gamma', 'highgamma', 'theta', 'clean')
for mode in modes:
    mode_corr_array = []
    for sub_num in range(1, 10):
        pred_pth = search_trf_files(sub_num, task='decode', mode=mode)[0]
        pred = np.load(pred_pth)
        pred = stat_utils.cutoff_or_pad(pred, len(GT))
        corr = stat_utils.componentwise_spearman(pred, GT)
        mode_corr_array.append(corr)
    mode_corr_array = np.array(mode_corr_array)  # Shape: (subjects, features)